In [77]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point

df = pd.read_excel('data/world_air_quality.xlsx')

In [78]:
df.head() 

,Country Code,City,Location,Coordinates,Pollutant,Source Name,Unit,Value,Last Updated,Country Label
0,DE,Moselle,FR22054,"49.1874042196733, 6.91086524487126",PM10,EEA France,µg/m³,6.60000,2023-10-31T02:00:00+00:00,Germany
1,DE,Niedersachsen,DENI051,"51.75816, 10.61248",NO,EEA Germany,µg/m³,0.14721,2023-10-31T04:00:00+00:00,Germany
2,DE,Sachsen,DESN001,"50.570872, 12.997278",NO,EEA Germany,µg/m³,1.00000,2023-10-31T04:00:00+00:00,Germany
3,DE,Baden-Württemberg,DEBW052,"47.664361, 9.169289",PM2.5,EEA Germany,µg/m³,2.00000,2023-10-31T04:00:00+00:00,Germany
4,DE,Baden-Württemberg,DEBW080,"49.00796, 8.38719",PM10,EEA Germany,µg/m³,4.30000,2023-10-31T04:00:00+00:00,Germany


In [79]:
# Tratamento de dados
print(df.isnull().sum()) # Verificar valores nulos
print(df.dtypes) # Alterar Last Update para datetime, Coordinates para geometry
# Verificar valores neagtivos em "Value"
print(df[df['Value'] < 0])

Country Code         0
City             23180
Location             2
Coordinates        219
Pollutant            0
Source Name          0
Unit                 0
Value                0
Last Updated         0
Country Label      115
dtype: int64
Country Code      object
City              object
Location          object
Coordinates       object
Pollutant         object
Source Name       object
Unit              object
Value            float64
Last Updated      object
Country Label     object
dtype: object
      Country Code                             City            Location  \
8               DE                    Niedersachsen             DENI028   
193             DE              Nordrhein-Westfalen             DENW059   
196             DE              Nordrhein-Westfalen             DENW053   
482             US                        Knoxville            LOUDONMS   
568             US                        CA8 - ARB           MMCA81006   
...            ...                         

In [80]:
# Descartar as linhas de coordenadas nulas
df = df.dropna(subset=['Coordinates'])
# Descartar as linhas de valores negativos
df = df[df['Value'] >= 0]

In [81]:
# Calcular o Intervalo Interquartil (IQR)
Q1 = df['Value'].quantile(0.25)
Q3 = df['Value'].quantile(0.75)
IQR = Q3 - Q1

# Definir limites para identificar outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Adicionar coluna "Outlier" ao DataFrame indicando se cada entrada é um outlier
df['Outlier'] = (df['Value'] < lower_bound) | (df['Value'] > upper_bound)
lower_limit = df['Value'].quantile(0.01)
upper_limit = df['Value'].quantile(0.99)

df['Value'] = np.where(df['Value'] < lower_limit, lower_limit, df['Value'])
df['Value'] = np.where(df['Value'] > upper_limit, upper_limit, df['Value'])


# Exibir as linhas com os valores que são outliers
outliers = df[df['Outlier']]
print("Outliers:")
print(outliers['Value'])

Outliers:
7          75.84000
10        256.65600
24        100.00000
40        500.00000
61         77.58907
            ...    
50062    1000.00000
50068      73.00000
50249     239.74000
50251     145.01000
50254     215.39001
Name: Value, Length: 6349, dtype: float64


In [82]:
mediana_pais = df.groupby('Country Label')['Value'].median().reset_index()
mediana_pais.columns = ['Country Label', 'Median']

In [83]:
# Iterar sobre os outliers e substituir os valores pela mediana do país, se disponível
for index, row in outliers.iterrows():
    pais = row['Country Label']
    
    # Verificar se o país está presente na tabela de medianas
    if pais in mediana_pais['Country Label'].values:
        valor_mediana = mediana_pais[mediana_pais['Country Label'] == pais]['Median'].values[0]
        df.loc[index, 'Value'] = valor_mediana
    else:
        # Se o país não estiver na tabela de medianas, substituir pelo valor mediano global
        mediana_global = df['Value'].median()
        df.loc[index, 'Value'] = mediana_global


In [84]:
# Tratamento de colunas com valores nulos
df['City'].fillna('Não informado', inplace=True)
df['Location'].fillna('DESC000', inplace=True)
df['Country Label'].fillna('Não informado', inplace=True)

In [85]:
# Converter coluna Last Update para datetime
df['Last Updated'] = pd.to_datetime(df['Last Updated'])

In [86]:
# Dividir coluna Coordinates em Latitude e Longitude
df['Latitude'] = df['Coordinates'].apply(lambda x: float(x.split(',')[0]))
df['Longitude'] = df['Coordinates'].apply(lambda x: float(x.split(',')[1]))

In [87]:
# Criar coluna geometry
df['geometry'] = None

for index, row in df.iterrows():
    df.loc[index, 'geometry'] = Point(row['Longitude'], row['Latitude'])

In [88]:
# Seleção de colunas necessárias
df = df[['City', 'Pollutant', 'Value', 'Unit', 'Last Updated', 'Country Label','Latitude', 'Longitude' ,'geometry']]

In [89]:
# Converter para GeoDataFrame
df = gpd.GeoDataFrame(df, geometry='geometry')

In [90]:
# Exportação para geojson
df.to_file('data/processado/geo_data.geojson', driver='GeoJSON')